In [6]:
using PyCall
using DataFrames

# Import the fine_tune_model and parse_decoded_strings functions from the Python script
py"""
import sys
sys.path.append(".")
from SGS_Tokenizer import ExtendedGPT2Tokenizer, GPT2Tokenizer, GPT2LMHeadModel, get_tensor_values
"""

# Example usage
vocab_file = "JLD2/vocab.json"
# Path to the vocab file
merges_file = "JLD2/merges.txt"  # Path to the merges file
tokenizer = py"ExtendedGPT2Tokenizer"(vocab_file, merges_file, 
            tensor_0_path="JLD2/tensor_0.pt", 
            tensor_1_path="JLD2/tensor_1.pt", 
            tensor_2_path="JLD2/tensor_2.pt", 
            tensor_3_path="JLD2/tensor_3.pt", 
            p=4)

# Example usage
# tokenizer.print_tensors()

text = "Import the fine_tune_model and parse_decoded_strings functions from the Python script"

# Update tensors
token_ids = tokenizer.tokenize_text(text, vocab_file, merges_file)  # Example token IDs
# println(token_ids)
hll_sha1, hll_id, token_hashes = tokenizer.update_tensors(token_ids)

println("token_hashes: ", hll_sha1, "; ", hll_id, "; ", token_hashes )

tokenizer.save_tensors("JLD2")

# Print updated tensors
# tokenizer.print_tensors()

println(tokenizer.tensor_sha1(tokenizer.tensor_3))

# Read all hashes from tensor_3 with known bin = 0, zeros = 1 using the Python function
bin = 1
zeros = 0
hashes = py"get_tensor_values"(tokenizer.tensor_3, bin, zeros)
println("All hashes at tensor_3[zeros=1, bin=0]: ", hashes)

# Print specific values to verify consistency
println("tensor_1 values: ", tokenizer.tensor_1)
println("tensor_3 values: ", tokenizer.tensor_3)


new_tensor_3[0, 1, 0] = 341032221
new_tensor_3[0, 4, 1] = 1311239075
new_tensor_3[0, 1, 2] = 486163617
new_tensor_3[2, 14, 3] = 3758668940
new_tensor_3[0, 10, 4] = 2912463139
new_tensor_3[0, 14, 5] = 3822972839
new_tensor_3[2, 14, 6] = 3758668940
new_tensor_3[0, 14, 7] = 3952959095
new_tensor_3[0, 11, 8] = 3166059063
new_tensor_3[0, 6, 9] = 1680850489
new_tensor_3[2, 14, 10] = 3758668940
new_tensor_3[1, 12, 11] = 3263161246
new_tensor_3[0, 14, 12] = 3911021279
new_tensor_3[2, 14, 13] = 3758668940
new_tensor_3[3, 13, 14] = 3726170216
new_tensor_3[2, 13, 15] = 3677716580
new_tensor_3[0, 2, 16] = 607936179
new_tensor_3[0, 4, 17] = 1311239075
new_tensor_3[0, 0, 18] = 166309807
new_tensor_3[0, 4, 19] = 1149258227
tensor_3 shape: torch.Size([32, 16, 64])
new_tensor_3 shape: torch.Size([32, 16, 20])
tensor_3 dtype: torch.int64
new_tensor_3 dtype: torch.int64
tensor_1: tensor([[     20939,  341032221],
        [       262, 1311239075],
        [      3734,  486163617],
        [        62, 375